In [4]:
import sys
import os
from pathlib import Path

# Get absolute path to project root
project_root = Path(os.getcwd()).resolve()
if project_root.name != "ncg87-blockchain_tracker":
    project_root = project_root.parent.parent  # Adjust if running from a subdirectory

sys.path.append(str(project_root))

# Add to Python path if not already there
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from config import Settings
from chains import XRPQuerier, XRPProcessor, XRPPipeline
import logging
from database import MongoDatabase, MongoInsertOperations, MongoQueryOperations

logging.basicConfig(level=logging.INFO,
                   format='%(asctime)s - %(levelname)s - %(message)s')

from decoder import LogDecoder
from config import Settings
from web3 import Web3



In [5]:
def decode_hex(value):
    """
    Decode a hexadecimal string to an integer if it's an Ethereum-style integer (e.g., block numbers, gas values).
    Does not decode long hashes or other non-integer hex values.
    :param value: Hexadecimal string (e.g., '0x677df92f') or other types.
    :return: Decoded integer or original value if not a valid short hex integer.
    """
    if isinstance(value, str) and value.startswith("0x"):
        if value == '0x':
            return None
        
        # Only decode if the hex string is short (e.g., block numbers, gas, timestamps)
        return int(value, 16)
    return value  # Return original value if not a short hex integer

# Parallelization of the tasks
def reconcile_logs_with_transactions(block):
    
    transactions = block["transactions"]
    w3 = Web3(Web3.HTTPProvider(Settings.ETHEREUM_ENDPOINT))
    
    # Fetch logs for the block
    logs = w3.eth.get_logs({
        "fromBlock": decode_hex(block['number']),
        "toBlock": decode_hex(block['number'])
    })

    # Group logs by transaction hash
    logs_by_transaction = {}
    for log in logs:
        tx_hash = log["transactionHash"].to_0x_hex()
        if tx_hash not in logs_by_transaction:
            logs_by_transaction[tx_hash] = []
        logs_by_transaction[tx_hash].append(dict(log))

    # Analyze the results
    tx_with_logs = len(logs_by_transaction)
    tx_without_logs = len(transactions) - tx_with_logs

    
    
    print(f"Total transactions in block: {len(transactions)}")
    print(f"Transactions with logs: {tx_with_logs}")
    print(f"Transactions without logs: {tx_without_logs}")

    # Return grouped logs for further analysis
    return logs_by_transaction



In [10]:
mongodb = MongoDatabase()
mongodb_query_ops = MongoQueryOperations(mongodb)

query = mongodb_query_ops.get_block_by_number(21585277, 'Ethereum')
data = query['raw_block_data']
logs_by_transaction = reconcile_logs_with_transactions(data)

2025-01-10 00:42:23,173 - INFO - Connected to MongoDB database: blockchain
2025-01-10 00:42:23,198 - INFO - Indexes created for collection: Solana
2025-01-10 00:42:23,200 - INFO - Indexes created for collection: Bitcoin
2025-01-10 00:42:23,201 - INFO - Indexes created for collection: Ethereum
2025-01-10 00:42:23,203 - INFO - Indexes created for collection: BNB
2025-01-10 00:42:23,205 - INFO - Indexes created for collection: XRP
2025-01-10 00:42:23,207 - INFO - Retrieved block 21585277 from Ethereum collection in MongoDB.


Total transactions in block: 91
Transactions with logs: 49
Transactions without logs: 42


In [11]:
log_decoder = LogDecoder(Settings.ETHERSCAN_API_KEY)

results = log_decoder.process_transaction_logs(logs_by_transaction)

Contract: 0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48, Event: ddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef, Count: 2
Contract: 0xdCD48419bD5Cd9d1b097695F2af4Ee125aADF84F, Event: 2a08a2bd2798f0aae9a843f0f4ad4de488c1b3d5f04049940cfed736ad69fb97, Count: 1
Contract: 0xc3CA38091061e3E5358A52d74730F16C60cA9c26, Event: 36c96ccc320eab9b6da1de6cc1e8ebc52a51cb114710b9c7db204cdad3067ce1, Count: 2
Contract: 0x9a15bB3a8FEc8d0d810691BAFE36f6e5d42360F7, Event: 4d8aead3491b7eba4b5c7a65fc17e493b9e63f9e433522fc5f6a85a168fc9d36, Count: 1
Contract: 0x5c7BCd6E7De5423a257D81B442095A1a6ced35C5, Event: a123dc29aebf7d0c3322c8eeb5b999e859f39937950ed31056532713d0de396f, Count: 1
Contract: 0xce16F69375520ab01377ce7B88f5BA8C48F8D666, Event: 5844b8bbe3fd2b0354e73f27bfde28d2e6d991f14139c382876ec4360391a47b, Count: 1
Contract: 0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48, Event: 8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925, Count: 1
Contract: 0x88e6A0c2dDD26FEEb64F039a2c41296FcB3f